<style>
    @media print{
        body {
            position:relative !important;
        }
        .celltag_new_page {
            page-break-before: always !important;
        }
    }
</style>
# COMPSCI 371D Homework 4

### Problem 0 (3 points)

## Part 1: Hyperplanes 

In [1]:
import pickle


filename = 'data.pkl'
with open(filename, 'rb') as file:
    data = pickle.load(file)

In [2]:
def check(P, n, beta):
    small = np.sqrt(np.finfo(float).eps)
    print('Checks for n = {}, beta = {}:'.format(n, beta))
    nn = np.linalg.norm(n)
    if np.abs(nn - 1.) < small:
        print('\tnorm of n is 1 as expected', end=', ')
    else:
        print('\tnorm of n is {}, while expected norm is 1'.format(nn))
    if beta >= 0:
        print('beta is nonnegative as expected')
    else:
        print('beta = {} is negative, while it should be nonnegative'.format(beta))
    if P.shape[0] == len(n):
        print('\tnumber of points is correct', end=', ')
    else:
        print('found {} points but expected {}'.format(P.shape[0], len(n)))
    checks = ', '.join(['OK' if ok else 'Not OK' for ok in
                        np.abs(np.dot(P, n) - beta) < small])
    print('individual point checks: {}'.format(checks))

### Problem 1.1 (Exam Style)

It is because the centroid of the hyperplane H is an average of all the known points $p_0$ thru $p_{j-1}$ that we know the centroid must be on the hyperplane using the fact that the sum and difference of any two points is a vector parallel to the surface on which they rest. Knowing this fact and noting that the only other factor is a division by a scalar, we know that the centroid must be located on the hyperplane as well.

### Problem 1.2 (Exam Style)

In [3]:
from scipy.linalg import null_space
import numpy as np

def null(A):
    return null_space(A).transpose()

In [4]:
def rank(A):
    row_num, col_num = A.shape
    null_row, null_col = null(A).shape
    return col_num -null_row

In [5]:
try:
    ranks = [str(rank(P)) for P in data['rank']]
    print(', '.join(ranks))
except NameError:
    print('rank not implemented?')

1, 2, 1, 2, 2, 3, 2


### Problem 1.3 (Exam Style)

In [6]:
def generic(P):
    j, d = P.shape
    if j > d+1:
        return False
    centroid = P.mean(0)
    ret = np.subtract(P, centroid)
    if rank(ret) == j-1:
        return True
    else:
        return False

In [7]:
try:
    answers = [str(generic(P)) for P in data['generic']]
    print(', '.join(answers))
except NameError:
    print('generic not implemented?')

True, True, True, False, False, False


### Problem 1.4 (Exam Style)

In [8]:
def hyperplane(P):
    j, d = P.shape
    assert (j == d and d > 0 and generic(P)), "Matrix not generic or wrong shape" 
    centroid = P.mean(0)
    hyper = np.subtract(P, centroid)
    Null = null(hyper)
    n = Null[0]
    beta = np.dot(n.T, P[0])
    if beta < 0:
        beta = beta * -1
        n = n * -1
    # n unit vector, beta nonnegative
    return n, beta
    

In [9]:
try:
    with np.printoptions(precision=4):
        for P in data['hyperplane']:
            n, beta = hyperplane(P)
            check(P, n, beta)
except NameError:
    print('hyperplane not implemented?')

Checks for n = [-1.], beta = 4.0:
	norm of n is 1 as expected, beta is nonnegative as expected
	number of points is correct, individual point checks: OK
Checks for n = [-0.7071  0.7071], beta = 1.1102230246251565e-16:
	norm of n is 1 as expected, beta is nonnegative as expected
	number of points is correct, individual point checks: OK, OK
Checks for n = [0.866 0.5  ], beta = 2.999999999999999:
	norm of n is 1 as expected, beta is nonnegative as expected
	number of points is correct, individual point checks: OK, OK
Checks for n = [-0.5774 -0.5774 -0.5774], beta = 2.0:
	norm of n is 1 as expected, beta is nonnegative as expected
	number of points is correct, individual point checks: OK, OK, OK
Checks for n = [0. 0. 0. 1.], beta = 0.0:
	norm of n is 1 as expected, beta is nonnegative as expected
	number of points is correct, individual point checks: OK, OK, OK, OK


### Problem 1.5 (Exam Style)

In [59]:
def points(n, beta):
    x_0 = n * beta    
    Null = null([n])
    P = np.add(Null, x_0)
    newP = np.vstack((P, x_0))
    return newP

We rearrage $n.T * x_0 = \beta$ using linear algebra to get 

In [60]:
try:
    with np.printoptions(precision=4):
        for n, beta in data['points']:
            P = points(n, beta)
            check(P, n, beta)
except NameError:
    print('points not implemented?')

Checks for n = [-1.], beta = 4.0:
	norm of n is 1 as expected, beta is nonnegative as expected
	number of points is correct, individual point checks: OK
Checks for n = [-0.7071  0.7071], beta = 0.0:
	norm of n is 1 as expected, beta is nonnegative as expected
	number of points is correct, individual point checks: OK, OK
Checks for n = [0.866 0.5  ], beta = 3.0:
	norm of n is 1 as expected, beta is nonnegative as expected
	number of points is correct, individual point checks: OK, OK
Checks for n = [-0.5774 -0.5774 -0.5774], beta = 2.0:
	norm of n is 1 as expected, beta is nonnegative as expected
	number of points is correct, individual point checks: OK, OK, OK
Checks for n = [0. 0. 0. 1.], beta = 0.0:
	norm of n is 1 as expected, beta is nonnegative as expected
	number of points is correct, individual point checks: OK, OK, OK, OK


## Part 2: A Binary Linear Classifier for Linearly Separable Data

### Problem 2.1 (Exam Style)

Differentiate the loss function with respect to w and b.

$[-y_n x_n] [-y_n]$

simplifies to

$-y_n [x_n] [1]$

### Problem 2.2

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

def plot_line(v):
    x_lim, y_lim = plt.xlim(), plt.ylim()
    w, b = v[:2], v[2]
    p = []
    if w[1] != 0.:
        for x in x_lim:
            y = - (b + w[0] * x) / w[1]
            if y_lim[0] <= y <= y_lim[1]:
                p.append((x, y))
    if w[0] != 0.:
        for y in y_lim:
            x = - (b + w[1] * y) / w[0]
            if x_lim[0] < x < x_lim[1]:
                p.append((x, y))
    if len(p) < 2:
        print('Warning: line does not intersect the axis rectangle')
    else:
        plt.plot([p[0][0], p[1][0]], [p[0][1], p[1][1]], 'r')
        plt.xlim(x_lim)
        plt.ylim(y_lim)


def show(T, v=None, epochs=None):
    x, y = (np.array(item) for item in list(zip(*T)))
    assert x.shape[1] == 2, 'Can only plot points on the plane'
    plt.plot(x[:, 0][y == -1], x[:, 1][y == -1], 'g^')
    plt.plot(x[:, 0][y == 1], x[:, 1][y == 1], 'bs')
    if v is not None:
        plot_line(v)
    plt.gca().set_aspect(1)
    plt.xticks([])
    plt.yticks([])
    if epochs is not None:
        plt.title('{} epochs to convergence'.format(epochs))

In [ ]:
import pickle


file_name = 'training_set.pkl'
with open(file_name, 'rb') as file:
    T = pickle.load(file)

In [ ]:
from random import shuffle

def train(T, r=.1, max_epochs=100):
    v = np.zeros(3, dtype=float)
    for i in range(0, max_epochs):
        shuffle(T)
        p = True
        for x, y in T:
            product = np.sign(np.dot(v[:2], x) + v[2])
            if product != y:
                p = False
                v = v + r * y * np.append(x, 1.0)
        if p == True:  
            break
            
    return v, i+1

print(train(T, r=.1, max_epochs=100))
for i in range(0,4):
    v, epochs = train(T, r=.1, max_epochs=100)
    show(T, v, epochs)
    plt.show()

Observations: Because the training data is shuffled each time, the boundary is found in a different order each time and therefore can be drawn in a number of different ways. Additionally, it may take more or less epochs to converge for the same reason. In general, it also converges pretty fast regardless.

### Problem 2.3 (Exam Style)

Example 1: $b_1 = b_2$ . Let $b_1 = b_2 = -1$. Let $w_1 = 2$ and $w_2 = 4$. Loss function would be 1 for $l_1$ and 3 for $l_2$. Hyperplane $- \frac{b}{w}$. So $x = \frac{1}{2}$ and $x = \frac{1}{4}$. Example 2: $w_1 = 2$, $w_2 = 4$; $b_1= 0$, $b_2 = 0$. Loss would be $l_1$ 2 and $l_2$ would be 4. Hyperplane is $x = 0$ for both.

### Problem 2.4 (Exam Style)

In 2.2 code, we update parameters whenever there is a misclassified point. But, if data is not linearly separable, anytime we try to reclassify one point, other points would then be incorrect. So, it would never converge and never be linearly seperable.

### Problem 2.5 (Exam Style)


Loss function is always strictly decreasing and eventually becomes constant at 0. If we select a and b, $\phi (b) < \phi (a)$. Another point c yields $\phi (c) < \phi(b)$.

When will the search for a value 𝑐 for the inital bracketing triple succeed, and when will it fail?

$ \phi$ b and $\phi$ c must be equal to zero. Otherwise, it will fail.

When it succeeds, does the bracketing triple shrink below 𝜖 if enough iterations of line search are performed?

Yes. When bracketing triple succeeds, line search performs as usual because all conditions are satisfied.

If 𝛼̂ is the value of 𝛼 returned by line search (regardless of whether the search for an initial 𝑐 succeeds or fails), is 𝛼̂ always within 𝜖 from a point where the loss is zero? That is, is there always a point 𝛼0 such that 𝜙(𝛼0)=0 and |𝛼̂ −𝛼0|≤𝜖 ?

Yes.